# Correction Vietnamese local strict spelling 

- Tích hợp hàm trích xuất danh từ riêng và số thông minh hơn.

# Libraries

In [ ]:
import os
import re
import sys
import gc


import signal
import logging
import pandas as pd
import torch
import unicodedata
import difflib


from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    GenerationConfig,
)
from dotenv import load_dotenv
from tqdm.notebook import tqdm
from huggingface_hub import snapshot_download


# Suppress Transformers Warning

In [ ]:
logging.getLogger("transformers.generation.utils").setLevel(logging.ERROR)


# LOAD .env 

In [ ]:
dotenv_path = os.path.join(os.getcwd(), "..", "envs", ".env")
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)
else:
    load_dotenv()

print(f"dotenv_path: {dotenv_path}")


dotenv_path: /home/guest/Projects/DSC2025/BAN/preprocess/../envs/.env


In [ ]:
HF_TOKEN = (
    os.getenv("HUGGING_FACE_TOKEN")
    or os.getenv("HUGGINGFACE_TOKEN")
    or os.getenv("HF_TOKEN")
    or os.getenv("HUNGGING_FACE_TOKEN")
    or None
)

if HF_TOKEN:
    print("OK!")
else:
    print("Add HF key to .env")


OK!


In [ ]:
public_test_correction_folder = "./vihallu_public_test_correction"
os.makedirs(public_test_correction_folder, exist_ok=True)
print(f"public_test_correction_folder_path: {public_test_correction_folder}")

debug_correction_villua_folder = "./debug_vihallu_public_test_correction"
os.makedirs(debug_correction_villua_folder, exist_ok=True)
print(f"debug_correction_villua_folder_path: {debug_correction_villua_folder}")


public_test_correction_folder_path: ./vihallu_public_test_correction
debug_correction_villua_folder_path: ./debug_vihallu_public_test_correction


# Config

In [ ]:
MODEL = "VietnamAIHub/Vietnamese_llama2_7B_8K_SFT_General_domain"
INPUT_CSV = "./vihallu-public-test.csv"
OUTPUT_CSV = os.path.join(
    public_test_correction_folder, "fixed-vihallu-public-test_final.csv"
)
COLUMNS_TO_FIX = ["context", "prompt", "response"]

MAX_INPUT_LENGTH = 2048
WORD_COUNT_TOLERANCE = 2
ACCEPT_SIMILARITY_THRESHOLD = 0.88
LENGTH_CHANGE_ALLOWED_RATIO = 0.15
STRICT_BASE_SIMILARITY = 0.96
LENIENT_BASE_SIMILARITY = 0.93


DEBUG_LOG_CSV = os.path.join(
    debug_correction_villua_folder, "debug_corrections_final.csv"
)
SAMPLE_DEBUG_N = 30


In [ ]:
FEW_SHOTS = [
    (
        "Nguồn thhu chinh của thuê truc tiếp la nhủg ai?",
        "Nguồn thu chính của thuế trực tiếp là những ai?",
    ),
    (
        "Ý nhĩa cũa viẹc tổ chưc cuôc thi Intervision là gì z?",
        "Ý nghĩa của việc tổ chức cuộc thi Intervision là gì?",
    ),
    ("thủ đô ha nội là trung tâm văn hóa", "thủ đô Hà Nội là trung tâm văn hóa"),
    (
        "...không đứng đầu thế giới về dự trữ ngoại tệ?",
        "...không đứng đầu thế giới về dự trữ ngoại tệ?",
    ),
    (
        "Các nhà khoa học muốn tìm đến S.J thì phải đến đâu?",
        "Các nhà khoa học muốn tìm đến S.J thì phải đến đâu?",
    ),
    (
        "...cuộc thi hát nổi tiếng thường niên...",
        "...cuộc thi hát nổi tiếng thường niên...",
    ),
]

for idx, few_shot in enumerate(FEW_SHOTS):
    print(f"index: {idx}\n+ Original: {few_shot[0]}\n-> Correction: {few_shot[1]}\n")


index: 0
+ Original: Nguồn thhu chinh của thuê truc tiếp la nhủg ai?
-> Correction: Nguồn thu chính của thuế trực tiếp là những ai?

index: 1
+ Original: Ý nhĩa cũa viẹc tổ chưc cuôc thi Intervision là gì z?
-> Correction: Ý nghĩa của việc tổ chức cuộc thi Intervision là gì?

index: 2
+ Original: thủ đô ha nội là trung tâm văn hóa
-> Correction: thủ đô Hà Nội là trung tâm văn hóa

index: 3
+ Original: ...không đứng đầu thế giới về dự trữ ngoại tệ?
-> Correction: ...không đứng đầu thế giới về dự trữ ngoại tệ?

index: 4
+ Original: Các nhà khoa học muốn tìm đến S.J thì phải đến đâu?
-> Correction: Các nhà khoa học muốn tìm đến S.J thì phải đến đâu?

index: 5
+ Original: ...cuộc thi hát nổi tiếng thường niên...
-> Correction: ...cuộc thi hát nổi tiếng thường niên...



# Device info

In [ ]:
print("CUDA available:", torch.cuda.is_available())
print("\n=== CONFIG: ===")
print(f"Model: {MODEL}")
print(f"INPUT CSV: {INPUT_CSV}")
print(f"OUTPUT CSV: {OUTPUT_CSV}")
print(f"DEBUG_LOG_CSV: {DEBUG_LOG_CSV}")

print(f"\n=== Columns to fix: {COLUMNS_TO_FIX} ===")
print(f"MAX_INPUT_LENGTH: {MAX_INPUT_LENGTH}")
print(f"WORD_COUNT_TOLERANCE: {WORD_COUNT_TOLERANCE}")
print(f"SMAMPLE_DEBUG_N:{SAMPLE_DEBUG_N}")

print("\n=== DEBUG: Thresholds (Final Optimized) ===")
print(f"STRICT (prompt, response): BASE_SIMILARITY >= {STRICT_BASE_SIMILARITY}")
print(f"LENGTH_CHANGE_ALLOWED_RATIO: {LENGTH_CHANGE_ALLOWED_RATIO}")
print(f"LENIENT (context): BASE_SIMILARITY >= {LENIENT_BASE_SIMILARITY}")
print(f"GENERAL: ACCEPT_SIMILARITY >= {ACCEPT_SIMILARITY_THRESHOLD}")
print("====================================\n")


CUDA available: True

=== CONFIG: ===
Model: VietnamAIHub/Vietnamese_llama2_7B_8K_SFT_General_domain
INPUT CSV: ./vihallu-public-test.csv
OUTPUT CSV: ./vihallu_public_test_correction/fixed-vihallu-public-test_final.csv
DEBUG_LOG_CSV: ./debug_vihallu_public_test_correction/debug_corrections_final.csv

=== Columns to fix: ['context', 'prompt', 'response'] ===
MAX_INPUT_LENGTH: 2048
WORD_COUNT_TOLERANCE: 1
SMAMPLE_DEBUG_N:30

=== DEBUG: Thresholds (Final Optimized) ===
STRICT (prompt, response): BASE_SIMILARITY >= 0.97
LENGTH_CHANGE_ALLOWED_RATIO: 0.15
LENIENT (context): BASE_SIMILARITY >= 0.94
GENERAL: ACCEPT_SIMILARITY >= 0.88



In [ ]:
if not os.path.exists(INPUT_CSV):
    raise FileNotFoundError(f"Không tìm thấy file '{INPUT_CSV}'.")

df = pd.read_csv(INPUT_CSV)
df_global = df


In [ ]:
# bộ chứa cho các row đã được xử lý (chỉ những row này sẽ được lưu vào OUTPUT_CSV)
processed_rows = []
processed_indices = set()
# STOP flag (sẽ được set True bởi signal handler)
stop_requested = False


# hàm lưu hiện trạng (chỉ lưu các cột index, context, prompt, response, predict_label)
def save_processed_rows_and_exit():
    if not processed_rows:
        print("No processed rows to save.")
        return
    output_df = pd.DataFrame(processed_rows)

    # đảm bảo đủ các cột cần thiết
    required_cols = ["index", "context", "prompt", "response", "predict_label"]
    for c in required_cols:
        if c not in output_df.columns:
            output_df[c] = ""

    output_df = output_df[required_cols]
    output_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print(f"Successfully saved {len(output_df)} processed rows to {OUTPUT_CSV}")


## Ctrl+C handler

In [ ]:
def signal_handler(sig, frame):
    global stop_requested
    print(
        "\n\n[Ctrl+C] Request received — will stop after current row and save processed rows..."
    )
    stop_requested = True


if "ipykernel" not in sys.modules and "IPython" not in sys.modules:
    signal.signal(signal.SIGINT, signal_handler)
else:
    print(
        "Detected IPython environment — relying on KeyboardInterrupt for interruption handling."
    )


Detected IPython environment — relying on KeyboardInterrupt for interruption handling.


## tokenizer/model setup -> set up 8-bit if not fallback to origin 16-bit

In [ ]:
tokenizer_kwargs = {"use_fast": False, "token": HF_TOKEN, "padding_side": "left"}


In [ ]:
def load_tokenizer_local_or_remote(model_id):
    try:
        tok = AutoTokenizer.from_pretrained(
            model_id,
            local_files_only=True,
            **{k: v for k, v in tokenizer_kwargs.items() if k != "token"}
        )
    except Exception:
        tok = AutoTokenizer.from_pretrained(model_id, **tokenizer_kwargs)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok


In [ ]:
def load_model_local_or_snapshot(model_id):
    cache_dir = os.path.expanduser("~/.cache/huggingface/hub")
    print(f"Cache huggingface path: {cache_dir}")
    loading_strategies = [
        # {"name": "8bit", "kwargs": {"quantization_config": BitsAndBytesConfig(load_in_8bit=True), "device_map": "auto", "token": HF_TOKEN, "cache_dir": cache_dir}},
        {
            "name": "4bit",
            "kwargs": {
                "quantization_config": BitsAndBytesConfig(
                    load_in_4bit=True,
                    bnb_4bit_quant_type="nf4",
                    bnb_4bit_compute_dtype=torch.bfloat16,
                ),
                "device_map": "auto",
                "token": HF_TOKEN,
                "cache_dir": cache_dir,
            },
        },
        {
            "name": "bfloat16",
            "kwargs": {
                "torch_dtype": torch.bfloat16,
                "device_map": "auto",
                "token": HF_TOKEN,
                "cache_dir": cache_dir,
            },
        },
    ]

    last_e = None
    for strat in loading_strategies:
        try:
            print(f"Trying to load with config: {strat['name']}")
            m = AutoModelForCausalLM.from_pretrained(model_id, **strat["kwargs"])
            print(f"Successfully loaded with config: {strat['name']}")
            model_path = (
                snapshot_download(model_id, token=HF_TOKEN, cache_dir=cache_dir)
                if HF_TOKEN
                else cache_dir
            )
            return m, model_path
        except Exception as e:
            print(f"Failed with config: {strat['name']} - {type(e).__name__}: {e}")
            last_e = e
    raise RuntimeError("Cannot load model") from last_e


## Load tokenizer

In [ ]:
print("Loading tokenizer...")
tokenizer = load_tokenizer_local_or_remote(MODEL)
tokenizer


Loading tokenizer...


LlamaTokenizer(name_or_path='VietnamAIHub/Vietnamese_llama2_7B_8K_SFT_General_domain', vocab_size=32000, model_max_length=8192, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}
)

## Load model

In [ ]:
print("Loading model...")
model, model_source = load_model_local_or_snapshot(MODEL)
model.eval()


Loading model...
Cache huggingface path: /home/guest/.cache/huggingface/hub
Trying to load with config: 4bit


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Successfully loaded with config: 4bit


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Lla

In [ ]:
try:
    model_device = next(model.parameters()).device
except:
    model_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Model device: {model_device}")
print(f"Model loaded from: {model_source}")


Model device: cuda:0
Model loaded from: /home/guest/.cache/huggingface/hub/models--VietnamAIHub--Vietnamese_llama2_7B_8K_SFT_General_domain/snapshots/12466d4619deed5fb5972760c082d7382151376c


# Utilities

## Remove diaccritics and punctuation

In [ ]:
def remove_diacritics_and_punct(s: str):
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = unicodedata.normalize("NFC", s)
    s = re.sub(r"[^\w\s]", "", s)
    return s.lower()


## Nomalize spaces

In [ ]:
def normalize_spaces(s: str):
    return re.sub(r"\s+", " ", s).strip()


# Calculate String Similarity

In [ ]:
def string_similarity(a, b):
    return difflib.SequenceMatcher(None, a, b).ratio()


## Extract number 

In [ ]:
def extract_numbers(text: str):
    """Trích xuất tất cả các chuỗi số, bao gồm cả số có dấu phẩy và dấu chấm."""
    return set(re.findall(r"\d+[,.]?\d*", text))


## Extract proper nouns

In [ ]:
def extract_proper_nouns(text: str):
    """
    Trích xuất các cụm danh từ riêng tiềm năng (chuỗi các từ viết hoa liền nhau).
    Hỗ trợ đầy đủ ký tự tiếng Việt và các trường hợp có dấu '.' hoặc '-'.
    """
    vietnamese_uppercase = (
        "A-ZÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝĂĐĨŨƠƯẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼẾỀỂỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪỬỮỰỲỴỶỸ"
    )
    pattern = (
        r"\b(["
        + vietnamese_uppercase
        + r"][\w\.-]+"
        + r"(?:\s+["
        + vietnamese_uppercase
        + r"][\w\.-]+)*)\b"
    )
    found_nouns = set(re.findall(pattern, text))
    common_acronyms = {"AI", "LLM", "UIT", "SED", "SKLP", "GDP", "UNESCO"}
    return {noun for noun in found_nouns if noun not in common_acronyms}


# LLM

## Prompt & Generation

In [ ]:
def make_correction_prompt_llama2(original_text):
    fs_examples = "\n".join(
        [f'Văn bản gốc: "{o}"\nVăn bản đã sửa: "{c}"' for o, c in FEW_SHOTS]
    )
    system_prompt = """
Bạn là một chuyên gia sửa lỗi chính tả tiếng Việt.  
Nhiệm vụ của bạn là kiểm tra và sửa lỗi chính tả của văn bản được cung cấp theo các quy tắc sau:

**QUY TẮC SỐ 1 (QUAN TRỌNG NHẤT):**  
Chỉ thay đổi những từ bị sai chính tả. Giữ nguyên các từ có nghĩa chuyên biệt (ví dụ: “ngoại tệ”) và giữ nguyên các từ viết tắt không rõ nghĩa (ví dụ: “S.J”).

**QUY TẮC SỐ 2:**  
Chỉ sửa lỗi chính tả, lỗi gõ phím, lỗi dấu thanh và lỗi viết hoa.

**QUY TẮC SỐ 3:**  
Nếu văn bản đã đúng, hãy lặp lại y hệt.

**QUY TẮC SỐ 4:**  
Không sửa dữ liệu số.
""".strip()
    user_prompt = (
        "Các ví dụ:\n"
        f"{fs_examples}\n\n"
        "Sửa văn bản sau. Chỉ trả về văn bản đã sửa.\n"
        f'Văn bản gốc: "{original_text}"'
    )
    return f'<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_prompt} [/INST] Văn bản đã sửa: "'


## Generate from model

In [ ]:
def generate_from_model(prompt, original_text_for_token_limit):
    inputs = tokenizer(
        prompt, return_tensors="pt", truncation=True, max_length=MAX_INPUT_LENGTH
    )
    try:
        inputs = {k: v.to(model_device) for k, v in inputs.items()}
    except Exception:
        pass

    input_token_length = len(
        tokenizer.encode(original_text_for_token_limit, add_special_tokens=False)
    )

    dynamic_min_new_tokens = int(input_token_length * 0.8)
    dynamic_max_new_tokens = int(input_token_length * 1.5) + 50

    gen_cfg = GenerationConfig(
        min_new_tokens=dynamic_min_new_tokens,
        max_new_tokens=dynamic_max_new_tokens,
        do_sample=True,
        num_beams=3,
        repetition_penalty=1.1,
        early_stopping=False,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    with torch.no_grad():
        out = model.generate(**inputs, generation_config=gen_cfg)
    full_decoded = tokenizer.decode(out[0], skip_special_tokens=True)

    if "[/INST]" in full_decoded:
        response_part = full_decoded.split("[/INST]")[-1].strip()
        if response_part.lower().startswith("văn bản đã sửa:"):
            response_part = response_part[len("Văn bản đã sửa:") :].strip()
        corrected = response_part.split("\n")[0].strip()
        corrected = re.sub(r'^["\']|["\']$', "", corrected)

        if corrected.endswith('"'):
            corrected = corrected[:-1]
        return corrected
    return ""


In [ ]:
text = "Dì hai lăng lọi đường xa đi tới thăm chú Tư"
generate_from_model(make_correction_prompt_llama2(text), text)


'Di chú hai lang loại đường xa đi để thăm Chùa'

# Correction function

In [ ]:
correction_cache = {}


In [ ]:
def correct_vietnamese_spelling(text, row_id, col_name, debug_print=False):
    original_text = "" if pd.isna(text) else str(text).strip()
    if not original_text:
        return text

    cache_key = (col_name, original_text)
    if cache_key in correction_cache:
        return correction_cache[cache_key]

    prompt = make_correction_prompt_llama2(original_text)
    corrected_text = generate_from_model(
        prompt, original_text_for_token_limit=original_text
    )

    accepted = False
    reason = "not_changed"
    final_text = original_text
    base_sim = 1.0

    # Chỉ check khi có sự thay đổi
    if corrected_text and corrected_text != original_text:
        passes_initial_checks = False

        current_base_threshold = (
            STRICT_BASE_SIMILARITY
            if col_name in ["prompt", "response"]
            else LENIENT_BASE_SIMILARITY
        )
        orig_words = normalize_spaces(original_text).split()
        corr_words = normalize_spaces(corrected_text).split()

        # Các điều kiện để accept hay reject cho llm generate để correct spelling
        base_sim = string_similarity(
            remove_diacritics_and_punct(original_text),
            remove_diacritics_and_punct(corrected_text),
        )

        if abs(len(orig_words) - len(corr_words)) > WORD_COUNT_TOLERANCE:
            reason = f"word_count_mismatch: original_words:{len(orig_words)} vs correction_word: {len(corr_words)}"
        elif base_sim < current_base_threshold:
            reason = f"base_similarity_too_low_{base_sim:.2f}_(threshold:{current_base_threshold})"
        elif (
            len(original_text) > 0
            and abs(len(corrected_text) - len(original_text)) / len(original_text)
            > LENGTH_CHANGE_ALLOWED_RATIO
        ):
            reason = "length_changed_too_much"
        elif (
            string_similarity(original_text, corrected_text)
            < ACCEPT_SIMILARITY_THRESHOLD
        ):
            reason = (
                f"low_similarity_{string_similarity(original_text, corrected_text):.2f}"
            )
        elif original_text.endswith("?") and not corrected_text.endswith("?"):
            reason = "question_mark_removed"
        else:
            passes_initial_checks = True

        # Nếu qua các điều kiện trên -> check thêm các lưới dự phòng => để tránh model generate ra những danh từ và số mới => dữ liệu càng bị hallucination
        if passes_initial_checks:
            original_numbers = extract_numbers(original_text)
            corrected_numbers = extract_numbers(corrected_text)
            original_nouns = extract_proper_nouns(original_text)
            corrected_nouns = extract_proper_nouns(corrected_text)

            # Kiểm tra sự tương đương tuyệt đối của các "sự thật"
            if original_numbers != corrected_numbers:
                reason = f"numbers_altered: {original_numbers.symmetric_difference(corrected_numbers)}"
                passes_initial_checks = False
            elif original_nouns != corrected_nouns:
                reason = f"proper_nouns_altered: {original_nouns.symmetric_difference(corrected_nouns)}"
                passes_initial_checks = False

        if passes_initial_checks:
            accepted = True
            reason = "accepted_change"
            final_text = corrected_text

    if debug_print:
        print("\n--- DEBUG SAMPLE ---")
        print(f"### ID: {row_id} | COLUMN: {col_name}")
        print(f"+   ORIGINAL: \n{original_text}")
        print(f"\n+ CORRECTED (model returned): \n{corrected_text}")

        if corrected_text and corrected_text != original_text:
            print("\n--- METRICS & THRESHOLDS ---")
            print(f"  - Base Similarity: {base_sim:.4f}")
            print(
                f"  - Direct Similarity: {string_similarity(original_text, corrected_text):.4f}"
            )

            print("\n--- FACT CHECKING ---")
            print(
                f"  - Original Numbers (len: {len(extract_numbers(original_text))}): \n{extract_numbers(original_text)}"
            )
            print(
                f"  - Corrected Numbers (len: {len(extract_numbers(corrected_text))}): \n{extract_numbers(corrected_text)}"
            )
            print(
                f"  - Original Nouns (len: {len(extract_proper_nouns(original_text))}): \n{extract_proper_nouns(original_text)}"
            )
            print(
                f"  - Corrected Nouns (len: {len(extract_proper_nouns(corrected_text))}): \n{extract_proper_nouns(corrected_text)}"
            )
        else:
            print("\n--- METRICS & THRESHOLDS ---")
            print("No change proposed by model.")

        print("\n--- DECISION ---")
        print(f"RESULT: {'ACCEPTED' if accepted else 'REJECTED'}")
        if not accepted:
            print(f"REASON: {reason}")
        print(f"FINAL TEXT: \n{final_text}")
        print("-" * 10)

    try:
        df_row = pd.DataFrame(
            [
                {
                    "id": row_id,
                    "column": col_name,
                    "original": original_text,
                    "corrected_model": corrected_text,
                    "final_text": final_text,
                    "accepted": accepted,
                    "reason": reason,
                    "similarity": (
                        string_similarity(original_text, corrected_text)
                        if corrected_text
                        else 1.0
                    ),
                    "base_similarity": base_sim,
                }
            ]
        )

        log_header = not os.path.exists(DEBUG_LOG_CSV)
        df_row.to_csv(
            DEBUG_LOG_CSV,
            index=False,
            mode="a",
            header=log_header,
            encoding="utf-8-sig",
        )
    except Exception as e:
        if debug_print:
            print(f"Warning: cannot write debug csv: {e}")

    correction_cache[cache_key] = final_text
    return final_text


# Main Loop for correcting Vietnamese's spelling

In [28]:
corrected_ids = set()
if os.path.exists(DEBUG_LOG_CSV):
    os.remove(DEBUG_LOG_CSV)

try:
    for index, row in tqdm(
        df.iterrows(),
        total=df.shape[0],
        desc="Processing",
        leave=True,
        dynamic_ncols=True,
    ):
        # nếu signal handler đã set stop_requested, thoát vòng lặp an toàn
        if stop_requested:
            print("\nStop requested — breaking main loop now.")
            break

        current_id = row.get("id", index)
        row_copy = row.copy()
        row_was_processed = False

        for col in COLUMNS_TO_FIX:
            if col not in row or pd.isna(row[col]):
                continue

            original_text = row[col]
            debug_flag = index < SAMPLE_DEBUG_N

            corrected_text = correct_vietnamese_spelling(
                original_text, current_id, col, debug_print=debug_flag
            )

            # đánh dấu row này đã được xử lý (đã cố gắng sửa ít nhất 1 cột)
            row_was_processed = True

            if str(original_text) != str(corrected_text):
                corrected_ids.add(current_id)
                row_copy[col] = corrected_text

        # nếu đã xử lý (đã thử sửa ít nhất 1 cột), lưu row vào processed_rows
        if row_was_processed:
            processed_indices.add(index)

            # chuẩn hóa trường predict_label nếu không tồn tại
            if "predict_label" not in row_copy.index:
                row_copy["predict_label"] = ""

            processed_rows.append(
                {
                    "index": index,
                    "context": row_copy.get("context", ""),
                    "prompt": row_copy.get("prompt", ""),
                    "response": row_copy.get("response", ""),
                    "predict_label": row_copy.get("predict_label", ""),
                }
            )

except KeyboardInterrupt:
    # trong notebook: Ctrl+C
    print("\nNgắt bởi người dùng. Đang lưu các hàng đã xử lý && giải phóng RAM/VRAM...")
    try:
        save_processed_rows_and_exit()
    except Exception as e:
        print(f"Error while saving: {e}")

except Exception as e:
    print(f"\nAn error occurred: {e}")

finally:
    # Khi hoàn tất (không có Ctrl+C), cũng chỉ lưu các hàng đã được xử lý
    save_processed_rows_and_exit()

    print("\n--- Summary ---")
    total_rows = df.shape[0]
    changed = len(corrected_ids)
    print(f"Đã xử lý tổng: {len(processed_rows)} hàng (trong tổng {total_rows}).")
    print(f"Có {changed} ids thay đổi.")

    # luôn giải phóng bộ nhớ ở cuối
    try:
        del df, df_global, corrected_ids
    except NameError:
        pass

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()


Processing:   0%|          | 0/1000 [00:00<?, ?it/s]


--- DEBUG SAMPLE ---
### ID: b709059b-b3b6-4ac2-bb88-2c794e2cc219 | COLUMN: context
+   ORIGINAL: 
Putin ngày 14 tháng 10 năm 2009, đưa ra đề nghị là Trung Quốc, các nước Trung Á và Nga nên tổ chức một cuộc thi hát hàng năm để có thể gia tăng các mối liên lạc văn hóa. Putin cũng đề nghị là cuộc thi hát này có thể được gọi là "Intervision" để đối đầu với cuộc thi hát nổi tiếng thường niên của lục địa châu Âu mang tên Eurovision. Một cuộc thi như vậy sẽ cho thấy các nam, nữ ca sĩ Trung Quốc tranh tài với các ca sĩ Uzbeek, Tadjik, Kazakh, Nga và Kyrgyzstan. Thông tấn xã Interfax tường thuật lời của Putin, nói thêm là: "Việc tổ chức một cuộc thi hát quốc tế hiện đại, Intervision, sẽ củng cố các mối liên lạc văn hóa giữa các nước chúng ta." Cuộc thi hát hàng năm của lục địa châu Âu, Eurovision, được khởi đầu từ năm 1956, đã thu hút mỗi lần cả trăm triệu khán giả truyền hình, không những của lục địa châu Âu, mà cả của thế giới nữa. Moskva cũng sau đó tổ chức cuộc thi hát Eurovision 2009 này